# Data Cleaning and Processing
- This small data set uses only 1 folder from the NYT Corpus [Articles from Jan 1 2007]
- This is to make sure that the script works before scaling up to the entire corpus

This script is mainly focusing on extracting and cleaning up the data. Then it will import the data into a pandas DataFrame. From there I can begin manipulate the data to take a form that will be more userful for doing the sentiment analysis.  
  
Because the NYT Corpus is already annotated and has a well-defined structure, cleaning is not as much of an issue as opposed to extracting the desired data for processing. 

----For Progress Report 2, I'm using my existing script and adding onto it. 

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [13]:
# import libraries
import xml.etree.ElementTree as Et
import glob
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

In [3]:
# create new dataframe with empty data
columns = ['DOCID', 'Date', 'Month', 'Year', 'Name', 'Text']
data = pd.DataFrame(columns=columns)

#### Append data to DataFrames
Immediately from the XML files, I want to retrieve data for the following columns: document ID, date, month, year, mentioned names, article text.  
This data is to be appended to a pandas DataFrame.

In [4]:
# open each xml file in the specified folder, open it and print out the names of mentioned people
for file in glob.glob("../data/NYT Corpus/nyt_corpus/data/2007/01/01/*.xml"):
    # parse the xml file into an element tree to extract data
    tree = Et.parse(file)
    root = tree.getroot()
    
    # get document id information (not sure if I need this yet, seems like it could be helpful)
    docid = root.find('.//doc-id[@id-string]').attrib['id-string']
    
    # get publication date information
    date = root.find(".//meta[@name='publication_day_of_month']").attrib['content']
    month = root.find(".//meta[@name='publication_month']").attrib['content']
    year = root.find(".//meta[@name='publication_year']").attrib['content']
    
    # get article text information
    # some articles seem to lack text - this is caught and handled in the if/else
    article = root.find(".//block[@class='full_text']/p")
    if article is not None:
        text = (article.text).lower()
    else:
        text = None
        
    # for each person mentioned, create a new row of data for them in the dataframe    
    for c in root.iter('person'):
        name = str(c.text).upper()
        data = data.append([{'DOCID': docid, 'Date': date, 'Month': month, 'Year': year, 'Name': name, 'Text': text}])
data.head()

,DOCID,Date,Month,Year,Name,Text
0,1815718,1,1,2007,"BLUMENTHAL, MARTIN",blumenthal--martin. a new york business man an...
0,1815719,1,1,2007,"BRADLEY, CAROL L.","bradley--carol l., 84, of tinton falls, nj die..."
0,1815720,1,1,2007,"CRAWFORD, PERRY JR.","crawford--perry jr., died at 89 on december 13..."
0,1815721,1,1,2007,"FLOOD, ROBERT FRANCIS","flood--robert francis, husband of the late cat..."
0,1815722,1,1,2007,"GEISLER, ENID (FRIEDMAN)","geisler--enid (friedman), on december 29, 2006..."


#### Start tweaking the DataFrame to make it more useful

In [5]:
# make the docid the index
data = data.set_index('DOCID')
data.head()

,Date,Month,Year,Name,Text
DOCID,,,,,
1815718,1,1,2007,"BLUMENTHAL, MARTIN",blumenthal--martin. a new york business man an...
1815719,1,1,2007,"BRADLEY, CAROL L.","bradley--carol l., 84, of tinton falls, nj die..."
1815720,1,1,2007,"CRAWFORD, PERRY JR.","crawford--perry jr., died at 89 on december 13..."
1815721,1,1,2007,"FLOOD, ROBERT FRANCIS","flood--robert francis, husband of the late cat..."
1815722,1,1,2007,"GEISLER, ENID (FRIEDMAN)","geisler--enid (friedman), on december 29, 2006..."


#### Create a new column with parsed text
This is to create a new column associated with each person that contains the text of the article they are mentioned in, but it is parsed using NLTK.word_tokenize. I can then use this column later for futher analysis.

In [6]:
# create function to tokenize the Text
def tokenizeText(col):
    return nltk.word_tokenize(str(col))

data['Tokenized'] = data['Text'].apply(tokenizeText)
data.head()

,Date,Month,Year,Name,Text,Tokenized
DOCID,,,,,,
1815718,1,1,2007,"BLUMENTHAL, MARTIN",blumenthal--martin. a new york business man an...,"[blumenthal, --, martin, ., a, new, york, busi..."
1815719,1,1,2007,"BRADLEY, CAROL L.","bradley--carol l., 84, of tinton falls, nj die...","[bradley, --, carol, l., ,, 84, ,, of, tinton,..."
1815720,1,1,2007,"CRAWFORD, PERRY JR.","crawford--perry jr., died at 89 on december 13...","[crawford, --, perry, jr., ,, died, at, 89, on..."
1815721,1,1,2007,"FLOOD, ROBERT FRANCIS","flood--robert francis, husband of the late cat...","[flood, --, robert, francis, ,, husband, of, t..."
1815722,1,1,2007,"GEISLER, ENID (FRIEDMAN)","geisler--enid (friedman), on december 29, 2006...","[geisler, --, enid, (, friedman, ), ,, on, dec..."


#### Right now, I want to be able to do the sentiment analysis on individual people
Once I can do sentiment analysis on individual people, I can further broaden that multiple people. When the script is later modified to process the entirety of the NYT corpus, I can aggregate frequent names and then analyze those names over time. Baby steps now I suppose.

In [27]:
# make a function that removes stopwords
def filter(toks):
    sw = set(stopwords.words('english'))
    others = ['--']
    filtered = [w for w in toks 
                if not w in sw 
                if not w in string.punctuation 
                if not w in others
               ]
    return filtered


t = data.iloc[0]['Tokenized']
filter(t)

['blumenthal',
 'martin',
 'new',
 'york',
 'business',
 'man',
 'philanthropist',
 'died',
 'last',
 'saturday',
 'manhattan',
 'long',
 'illness',
 '90.',
 'mr.',
 'blumenthal',
 'born',
 'frankfurt',
 'germany',
 'immigrated',
 'new',
 'york',
 'city',
 '1935.',
 'president',
 'a.j',
 'hollander',
 'company',
 'commodities',
 'trading',
 'firm',
 'retirement',
 'devoted',
 'philanthropic',
 'activities',
 'trustee',
 'ymha',
 'served',
 'chairman',
 'bezalel',
 'charitable',
 'organization',
 'supports',
 'arts',
 'israel',
 'also',
 'active',
 'human',
 'rights',
 'watch',
 'mr.',
 'blumenthal',
 'survived',
 'wife',
 'sallie',
 'blumenthal',
 'children',
 'richard',
 'greenwich',
 'david',
 'boston',
 'six',
 'grandchildren',
 'brother',
 'fred',
 'sister',
 'edith',
 'levisohn',
 'first',
 'wife',
 'jane',
 'died',
 '1969.',
 'funeral',
 'take',
 'place',
 'riverside',
 'chapel',
 '11:30am',
 'tuesday',
 'january',
 '2nd',
 '6',
 'blumenthal',
 'martin',
 'park',
 'avenue',
 'syn

In [29]:
# map my new filter function to the dataframe
# this will clean up the Tokenized column to do work on
data["Tokenized"] = data["Tokenized"].apply(lambda x: filter(x))
data.head()

,Date,Month,Year,Name,Text,Tokenized
DOCID,,,,,,
1815718,1,1,2007,"BLUMENTHAL, MARTIN",blumenthal--martin. a new york business man an...,"[blumenthal, martin, new, york, business, man,..."
1815719,1,1,2007,"BRADLEY, CAROL L.","bradley--carol l., 84, of tinton falls, nj die...","[bradley, carol, l., 84, tinton, falls, nj, di..."
1815720,1,1,2007,"CRAWFORD, PERRY JR.","crawford--perry jr., died at 89 on december 13...","[crawford, perry, jr., died, 89, december, 13t..."
1815721,1,1,2007,"FLOOD, ROBERT FRANCIS","flood--robert francis, husband of the late cat...","[flood, robert, francis, husband, late, cather..."
1815722,1,1,2007,"GEISLER, ENID (FRIEDMAN)","geisler--enid (friedman), on december 29, 2006...","[geisler, enid, friedman, december, 29, 2006.,..."


## Future Processing Tasks
Further tasks to do is to process the tokenized values and strip out any stop words or words irrelevant to sentiment. I also need to look into bringing another corpus with has words associated with positive/negative/neutral sentiment in order to make decisions on the words that are remaining. Additionally, I am noticing duplicate names being entered, however they are slightly different strings and thus simply running set() won't remove them. I need a way to standardize the names so that they can fold into each other nicely.